In [1]:
!pip install azureml-train

In [2]:
pip install azureml-train-core

Note: you may need to restart the kernel to use updated packages.


In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-233945
Azure region: westeurope
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-233945


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                script='train.py',
                compute_target=cluster,
                environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6
Web View: https://ml.azure.com/runs/HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-233945/workspaces/quick-starts-ws-233945&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-05-18T12:07:09.860103][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-05-18T12:07:10.3119922Z][SCHEDULER][INFO]Scheduling job, id='HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6_0' 
[2023-05-18T12:07:10.4129569Z][SCHEDULER][INFO]Scheduling job, id='HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6_1' 
[2023-05-18T12:07:10.5221595Z][SCHEDULER][INFO]Scheduling job, id='HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6_2' 
[2023-05-18T12:07:10.6021616Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_dbc48dd6-2130-4226-ab3e-ba47108f28a6_0' 
[2023-05-18T12:07:10.6755312Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_dbc48dd6-2130-4226-ab3e-

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt. Marking the experiment as failed because initial child jobs have failed due to user error",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt. Marking the experiment as failed because initial child jobs have failed due to user error\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

No such comm: 54a164bed6dc4554a641e89272fe3f4b
No such comm: 96dc85044cd04d93bdfe1e80848685b1
No such comm: e8aa3c851734482e8ee84f1fabb55496
No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: 54a164bed6dc4554a641e89272fe3f4b


KeyError: 'log_files'

No such comm: 96dc85044cd04d93bdfe1e80848685b1
No such comm: e8aa3c851734482e8ee84f1fabb55496
No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: 54a164bed6dc4554a641e89272fe3f4b
No such comm: 96dc85044cd04d93bdfe1e80848685b1
No such comm: e8aa3c851734482e8ee84f1fabb55496
No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: 54a164bed6dc4554a641e89272fe3f4b
No such comm: 96dc85044cd04d93bdfe1e80848685b1
No such comm: e8aa3c851734482e8ee84f1fabb55496


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: 54a164bed6dc4554a641e89272fe3f4b
No such comm: 96dc85044cd04d93bdfe1e80848685b1
No such comm: e8aa3c851734482e8ee84f1fabb55496
No such comm: a16342d1817746c88d667ee2eb0fd006
No such comm: 54a164bed6dc4554a641e89272fe3f4b


In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.register_model(model_name = 'hd_best_run.pkl', model_path='outputs/')
print(best_run)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [ ]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, model = run.get_output()
best_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_run)